In [1]:
# ------------------------------------------------------------------------------------
# Going through all image-gt pairs and collect all dice scores with gamma + otsu local
# ------------------------------------------------------------------------------------

import numpy as np
import matplotlib.pyplot as plt
import os
import sys
from pathlib import Path
from skimage.io import imread
from typing import Union, Tuple

# Current folder as project_root
# Going three layers of folders upwards to src
project_root = os.path.abspath(os.path.join(os.getcwd(), "..", "..", ".."))
src_dir      = os.path.join(project_root)

# Add the src directory to the Python module search path
if src_dir not in sys.path:
    sys.path.insert(0, src_dir)

# Import functions from src
import importlib
import src.Dice_Score
importlib.reload(src.Dice_Score)

from src.pre_processing import gammacorrection
from src.Otsu_Local import local_otsu_package
from src.Dice_Score import dice_score

# Process single image and its groundtruth
def process_single(img_path: Path, gt_path: Path, s) -> float:
    """ 
    Reads one image and corresponding groundtruth, proccesses, segments and computes dice score for one image.
    """
    # Reads image and reads, binarizes groundtruth
    img = imread(img_path, as_gray=True)
    gt = imread(gt_path, as_gray=True)
    gt  = 1 - (gt == 0)         

    # Gamma correction
    img_gamma = gammacorrection(img, gamma=0.6)
    
    # local Otsu tresholding
    t = local_otsu_package(img_gamma, radius=s)

    # binarize
    binary = (img_gamma > t).astype(np.uint8)

    # 5. Compute Dice score
    return dice_score(binary.flatten(), gt.flatten())

# data set: NIH3T3
img_dir = Path(project_root) / "data-git" / "NIH3T3" / "img"
gt_dir = Path(project_root) / "data-git" / "NIH3T3" / "gt"

radius_dict = {
    0: 550,
    1: 500,
    26: 150,
    27: 100,
    28: 250,
    29: 150,
    30: 100,
    31: 100,
    32: 100,
    33: 150,
    37: 100,
    40: 200,
    42: 150,
    44: 150,
    45: 150,
    46: 150,
    47: 150,
    49: 150
}

dice_scores_NIH3T3 = []

for img_file in sorted(img_dir.glob("dna-*.png")):
    # Extract index to read the corresponding gt
    idx = img_file.stem.split('-')[-1]
    gt_file = gt_dir / f"{idx}.png"
    idx = int(idx)
    if not gt_file.exists():
        print(f" ! no Ground-Truth for {img_file.name} !skip …")
        continue
    
    r = radius_dict.get(idx)
    if not isinstance(r, int):
     raise ValueError(f"Invalid radius for index {idx}: {r}")

    score = process_single(img_file, gt_file, s=r)
    dice_scores_NIH3T3.append(score)
    print(f"{img_file.name:<20} Dice = {score:.4f}")

# Vector containing all dice scores
dice_scores_NIH3T3 = np.array(dice_scores_NIH3T3)           
print("\nDONE ⇢ Mean Dice:", dice_scores_NIH3T3.mean())

np.save('NIH3T3_dice_scores_gamma+local', dice_scores_NIH3T3)



# data set: N2DL-HeLa
img_dir = Path(project_root) / "data-git" / "N2DL-HeLa" / "img"
gt_dir = Path(project_root) / "data-git" / "N2DL-HeLa" / "gt"

radius_dict = {
    13: 700,
    52: 200,
    75: 50,
    79: 50
}

dice_scores_N2DL_HeLa = []

for img_file in sorted(img_dir.glob("t*.tif")):
    # Extract index to read the corresponding gt
    idx = img_file.stem.split('t')[-1]
    gt_file = gt_dir / f"man_seg{idx}.tif"
    idx = int(idx)

    if not gt_file.exists():
        print(f" ! no Ground-Truth for {img_file.name} !skip …")
        continue

    r = radius_dict.get(idx)

    score = process_single(img_file, gt_file, s=r)
    dice_scores_N2DL_HeLa.append(score)
    print(f"{img_file.name:<20} Dice = {score:.4f}")

# Vector containing all dice scores
dice_scores_N2DL_HeLa = np.array(dice_scores_N2DL_HeLa)           
print("\nDONE ⇢ Mean Dice:", dice_scores_N2DL_HeLa.mean())

np.save('N2DL-HeLa_dice_scores_gamma+local', dice_scores_N2DL_HeLa)


# data set: N2DH-GOWT1
img_dir = Path(project_root) / "data-git" / "N2DH-GOWT1" / "img"
gt_dir = Path(project_root) / "data-git" / "N2DH-GOWT1" / "gt"

radius_dict = {
    1: 350,
    21: 550,
    31: 450,
    39: 450,
    52: 900,
    72: 100
}

dice_scores_N2DH_GOWT1 = []

for img_file in sorted(img_dir.glob("t*.tif")):
    # Extract index to read the corresponding gt
    idx = img_file.stem.split('t')[-1]
    gt_file = gt_dir / f"man_seg{idx}.tif"
    idx = int(idx)

    if not gt_file.exists():
        print(f" ! no Ground-Truth for {img_file.name} !skip …")
        continue
    
    r = radius_dict.get(idx)

    score = process_single(img_file, gt_file, s=r)
    dice_scores_N2DH_GOWT1.append(score)
    print(f"{img_file.name:<20} Dice = {score:.4f}")

# Vector containing all dice scores
dice_scores_N2DH_GOWT1 = np.array(dice_scores_N2DH_GOWT1)           
print("\nDONE ⇢ Mean Dice:", dice_scores_N2DH_GOWT1.mean())

np.save('N2DH-GOWT1_dice_scores_gamma+local', dice_scores_N2DH_GOWT1)

dna-0.png            Dice = 0.9320
dna-1.png            Dice = 0.9337
dna-26.png           Dice = 0.8777
dna-27.png           Dice = 0.8261
dna-28.png           Dice = 0.8465
dna-29.png           Dice = 0.7712
dna-30.png           Dice = 0.7989
dna-31.png           Dice = 0.8471
dna-32.png           Dice = 0.6767
dna-33.png           Dice = 0.8130
dna-37.png           Dice = 0.6864
dna-40.png           Dice = 0.8217
dna-42.png           Dice = 0.7222
dna-44.png           Dice = 0.8597
dna-45.png           Dice = 0.8705
dna-46.png           Dice = 0.7481
dna-47.png           Dice = 0.7709
dna-49.png           Dice = 0.8668

DONE ⇢ Mean Dice: 0.8149794275826635
t13.tif              Dice = 0.7662
t52.tif              Dice = 0.8131
t75.tif              Dice = 0.8822
t79.tif              Dice = 0.8896

DONE ⇢ Mean Dice: 0.8377639899303756
t01.tif              Dice = 0.5991
t21.tif              Dice = 0.6120
t31.tif              Dice = 0.7383
t39.tif              Dice = 0.7518
t52.tif       

In [ ]:
import os
import sys
import numpy as np
from pathlib import Path
from skimage.io import imread
from typing import Callable, Dict, List, Tuple

# Setup project_root and ensure it's a Path
project_root = Path(os.path.abspath(os.path.join(os.getcwd(), "..", "..", "..")))
sys.path.insert(0, str(project_root))

import importlib
import src.Dice_Score
importlib.reload(src.Dice_Score)

from src.pre_processing import gammacorrection
from src.Otsu_Local import local_otsu_package
from src.Dice_Score import dice_score
from src.imread_all import (
    load_nih3t3_images,
    load_n2dl_hela_images,
    load_n2dh_gowt1_images,
)

# Single-image processing (unchanged)
def process_single(img_path: Path, gt_path: Path, radius: int) -> float:
    img = imread(img_path, as_gray=True)
    gt = imread(gt_path, as_gray=True)
    gt_bin = 1 - (gt == 0)
    img_gamma = gammacorrection(img, gamma=0.6)
    t = local_otsu_package(img_gamma, radius=radius)
    binary = (img_gamma > t).astype(np.uint8)
    return dice_score(binary.flatten(), gt_bin.flatten())

# Dataset configuration
DatasetConfig = Tuple[
    str,
    Callable[..., Tuple[List[np.ndarray], List[np.ndarray], List[str], List[str]]],
    Dict[int, int],
    Callable[[str], int]
]

datasets: List[DatasetConfig] = [
    (
        'NIH3T3',
        load_nih3t3_images,
        { 0:550,1:500,26:150,27:100,28:250,29:150,30:100,31:100,32:100,33:150,37:100,40:200,42:150,44:150,45:150,46:150,47:150,49:150 },
        lambda p: int(Path(p).stem.split('-')[-1])
    ),
    (
        'N2DL-HeLa',
        load_n2dl_hela_images,
        {13:700,52:200,75:50,79:50},
        lambda p: int(Path(p).stem.split('t')[-1])
    ),
    (
        'N2DH-GOWT1',
        load_n2dh_gowt1_images,
        {1:350,21:550,31:450,39:450,52:900,72:100},
        lambda p: int(Path(p).stem.split('t')[-1])
    ),
]

# Main evaluation loop
def evaluate_all_local(output_dir: Path = Path('Dice_scores')) -> None:
    output_dir.mkdir(exist_ok=True)

    for name, loader, radius_dict, idx_fn in datasets:
        # Build base path using os.path.join to avoid Path/str issues
        base_path = os.path.join(str(project_root), 'data-git', name)
        imgs, gts, img_paths, gt_paths = loader(base_path=base_path)

        scores: List[float] = []
        print(f"=== {name} ===")
        for img_p, gt_p in zip(img_paths, gt_paths):
            idx = idx_fn(img_p)
            if idx not in radius_dict:
                print(f" Skipping {Path(img_p).name}: no radius defined for idx {idx}")
                continue
            r = radius_dict[idx]
            score = process_single(Path(img_p), Path(gt_p), radius=r)
            scores.append(score)
            print(f"{Path(img_p).name}: Dice={score:.4f}")

        scores_arr = np.array(scores)
        print(f"Mean Dice ({name}): {scores_arr.mean():.4f}\n")
        
        return 


evaluate_all_local()


=== NIH3T3 ===
dna-0.png: Dice=0.9320
dna-1.png: Dice=0.9337
dna-26.png: Dice=0.8777
dna-27.png: Dice=0.8261
dna-28.png: Dice=0.8465
dna-29.png: Dice=0.7712
dna-30.png: Dice=0.7989
dna-31.png: Dice=0.8471
dna-32.png: Dice=0.6767
dna-33.png: Dice=0.8130
dna-37.png: Dice=0.6864
dna-40.png: Dice=0.8217
dna-42.png: Dice=0.7222
dna-44.png: Dice=0.8597
dna-45.png: Dice=0.8705
dna-46.png: Dice=0.7481
dna-47.png: Dice=0.7709
dna-49.png: Dice=0.8668
Mean Dice (NIH3T3): 0.8150

=== N2DL-HeLa ===
t13.tif: Dice=0.7662
t52.tif: Dice=0.8131
t75.tif: Dice=0.8822
t79.tif: Dice=0.8896
Mean Dice (N2DL-HeLa): 0.8378

=== N2DH-GOWT1 ===
t01.tif: Dice=0.5991
t21.tif: Dice=0.6120
t31.tif: Dice=0.7383
t39.tif: Dice=0.7518
t52.tif: Dice=0.6264
t72.tif: Dice=0.7226
Mean Dice (N2DH-GOWT1): 0.6750

